# HuggingFace Genomics Hands‑On Exercise
## DNA Sequence Analysis with Foundation Models

**Author:** Ikram Ullah, KAUST Bioinformatics Platform


## Setup (1/2) Install and import all required Python packages

We install PyTorch (CUDA 12.1 wheels if NVIDIA GPU is available), visualization and ML libraries, and Hugging Face tooling so the rest of the notebook runs without errors.

In [1]:
# Install packages
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
#!pip install matplotlib scikit-learn ipython 
#!pip install transformers umap-learn evaluate 
#!pip uninstall -y triton

In [2]:
# Import libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel, AutoConfig
from sklearn.decomposition import PCA
import umap
import time

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch: 2.6.0+cu124
CUDA available: True


## Setup (2/2)
**What & Why:** Select a compute device. We prefer GPU if available for speed; otherwise the code falls back to CPU automatically.

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using: {device}")

Using: cuda


## Part 1: PyTorch Tensors (1/3)
**What & Why:** Convert a DNA string into integer indices (A/T/C/G → 0/1/2/3). This is a simple way to map symbolic sequences into numeric tensors for downstream models.

In [4]:
dna = "ATCGATCGATCG"
mapping = {'A': 0, 'T': 1, 'C': 2, 'G': 3}

encoded = [mapping[b] for b in dna]
tensor = torch.tensor(encoded)

print(f"Original: {dna}")
print(f"Tensor: {tensor}")
print(f"Shape: {tensor.shape}")

Original: ATCGATCGATCG
Tensor: tensor([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3])
Shape: torch.Size([12])


## Part 1: PyTorch Tensors (2/3)
**What & Why:** One‑hot encode a DNA sequence (length × 4). One‑hot vectors are a common baseline representation for nucleotides and enable simple batching.

In [5]:
def one_hot_encode(seq):
    mapping = {'A': 0, 'T': 1, 'C': 2, 'G': 3}
    one_hot = torch.zeros(len(seq), 4)
    for i, b in enumerate(seq):
        one_hot[i, mapping[b]] = 1
    return one_hot

encoded = one_hot_encode("ATCG")
print(encoded.shape)  # (4, 4)

seqs = ["ATCG", "GCTA"]
batch = torch.stack([one_hot_encode(s) for s in seqs])
print(batch.shape)  # (2, 4, 4)

torch.Size([4, 4])
torch.Size([2, 4, 4])


## Part 1: PyTorch Tensors (3/3)
**What & Why:** Demonstrate moving tensors between CPU and GPU. Keeping model and tensors on the same device is essential for correct and fast computation.

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

t = torch.randn(100, 512)
t_gpu = t.to(device)
print("Device:", t_gpu.device)

t_cpu = t_gpu.cpu()
print("Back to:", t_cpu.device)

Device: cuda:0
Back to: cpu


## Part 2: Load Models (1/2) — DNABERT‑2
**What & Why:** Load DNABERT‑2 with `trust_remote_code=True` and its repo config. This avoids config‑class mismatches when custom remote code is used.

In [7]:
from transformers import AutoTokenizer, AutoModel, BertConfig
from termcolor import colored
from IPython.display import display, Markdown

model_name = "zhihan1996/DNABERT-2-117M"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
config = BertConfig.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True, config=config)

# Compute parameters
params = sum(p.numel() for p in model.parameters())

# Prepare styled Markdown output
info_md = f"""
# 🧬 **DNABERT-2 Model Summary**

| Property | Description |
|:--|:--|
| **Model ID** | `{model_name}` |
| **Architecture** | **BERT + ALiBi (Attention with Linear Biases)** |
| **Tokenizer Type** | `{type(tokenizer).__name__}` |
| **Vocab Size** | **{tokenizer.vocab_size:,}** |
| **Max Sequence Length** | **{config.max_position_embeddings}** |
| **Hidden Size** | **{config.hidden_size}** |
| **Number of Layers** | **{config.num_hidden_layers}** |
| **Attention Heads** | **{config.num_attention_heads}** |
| **Total Parameters** | 🧠 **{params:,}** |

> ℹ️ *Source:* [Hugging Face Model Card](https://huggingface.co/zhihan1996/DNABERT-2-117M)
"""

display(Markdown(info_md))

Some weights of BertModel were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



# 🧬 **DNABERT-2 Model Summary**

| Property | Description |
|:--|:--|
| **Model ID** | `zhihan1996/DNABERT-2-117M` |
| **Architecture** | **BERT + ALiBi (Attention with Linear Biases)** |
| **Tokenizer Type** | `PreTrainedTokenizerFast` |
| **Vocab Size** | **4,096** |
| **Max Sequence Length** | **512** |
| **Hidden Size** | **768** |
| **Number of Layers** | **12** |
| **Attention Heads** | **12** |
| **Total Parameters** | 🧠 **117,068,544** |

> ℹ️ *Source:* [Hugging Face Model Card](https://huggingface.co/zhihan1996/DNABERT-2-117M)


## Part 2: Load Models (2/2) — Nucleotide Transformer
**What & Why:** Load InstaDeep’s Nucleotide Transformer as an alternative foundation model. We’ll later compare model sizes and use either for embeddings.

In [8]:
nt_name = "InstaDeepAI/nucleotide-transformer-500m-human-ref"

# Load tokenizer and model
nt_tokenizer = AutoTokenizer.from_pretrained(nt_name, trust_remote_code=True)
nt_config = AutoConfig.from_pretrained(nt_name)
nt_model = AutoModel.from_pretrained(nt_name, trust_remote_code=True).to(device).eval()

# Count parameters
nt_params = sum(p.numel() for p in nt_model.parameters())

nt_md = f"""
# 🧬 **Nucleotide Transformer (Human-Ref, 500M) Summary**

| Property | Description |
|:--|:--|
| **Model ID** | `{nt_name}` |
| **Architecture** | **Transformer (BERT-like, RoFormer variant)** |
| **Pretraining Objective** | Masked Language Modeling (on reference genomes) |
| **Tokenizer Type** | `{type(nt_tokenizer).__name__}` |
| **Vocab Size** | **{nt_tokenizer.vocab_size:,}** |
| **Max Sequence Length** | **{nt_config.max_position_embeddings}** |
| **Hidden Size** | **{nt_config.hidden_size}** |
| **Number of Layers** | **{nt_config.num_hidden_layers}** |
| **Attention Heads** | **{nt_config.num_attention_heads}** |
| **Total Parameters** | 🧠 **{nt_params:,}** |
| **Training Data** | Human reference genome (GRCh38) + RefSeq |
| **Key Features** | Large-scale self-supervised pretraining, cross-species generalization |
| **Paper / Preprint** | [Biorxiv: *Nucleotide Transformers for Genomics*](https://www.biorxiv.org/content/10.1101/2023.01.11.523679v2) |

> ⚙️ *Implements a scalable Transformer backbone for genomics, trained on billions of nucleotides.*
"""

display(Markdown(nt_md))

Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



# 🧬 **Nucleotide Transformer (Human-Ref, 500M) Summary**

| Property | Description |
|:--|:--|
| **Model ID** | `InstaDeepAI/nucleotide-transformer-500m-human-ref` |
| **Architecture** | **Transformer (BERT-like, RoFormer variant)** |
| **Pretraining Objective** | Masked Language Modeling (on reference genomes) |
| **Tokenizer Type** | `EsmTokenizer` |
| **Vocab Size** | **4,107** |
| **Max Sequence Length** | **1002** |
| **Hidden Size** | **1280** |
| **Number of Layers** | **24** |
| **Attention Heads** | **20** |
| **Total Parameters** | 🧠 **480,438,241** |
| **Training Data** | Human reference genome (GRCh38) + RefSeq |
| **Key Features** | Large-scale self-supervised pretraining, cross-species generalization |
| **Paper / Preprint** | [Biorxiv: *Nucleotide Transformers for Genomics*](https://www.biorxiv.org/content/10.1101/2023.01.11.523679v2) |

> ⚙️ *Implements a scalable Transformer backbone for genomics, trained on billions of nucleotides.*


## Part 3: Hugging Face Platform
**What & Why:** Explore https://huggingface.co/models to discover genomic models (e.g., DNABERT‑2 and Nucleotide Transformer). Read model cards to learn tokenization, max sequence length, architecture, and intended uses.

## Part 3: Hugging Face Platform (Browse the Model Hub)

* **Visit** → https://huggingface.co/models

* **Search examples**:
    * "DNABERT" → zhihan1996/DNABERT-2-117M
    * "nucleotide transformer" → InstaDeepAI variants
    * "genomic" → multiple models
* **Explore**:
    * Model ID
    * Download count
    * Tokenization method

## Part 3: Hugging Face Platform (Exercise 2.2: Model Card Analysis)

* **Visit** → [DNABERT-2 Model Card](https://huggingface.co/zhihan1996/DNABERT-2-117M)
* **Find**
    * Tokenization → BPE
    * Max seq length → 512
    * Architecture → BERT + ALiBi
    * Parameters → 117M
* **Model Card Sections**
    * Overview
    * Model Details

## Part 4: Hugging Face Inference API (1/3) — Setup Authentication
**What & Why:** Store your personal access token (read access) to call the hosted Inference API for models without local setup.

In [ ]:
HF_TOKEN = "your_token_here"  # Replace with your token from https://huggingface.co/settings/tokens

## Part 4: Hugging Face Inference API (2/3) — Query the API
**What & Why:** Define a helper that posts a sequence to the Inference API and returns JSON output. This is handy for quick prototypes or when GPUs aren’t available locally.

In [10]:
import requests
import pandas as pd
from IPython.display import display, HTML

def query_api(seq, model_id, token):
    """
    Query the Hugging Face Inference API for a given model and DNA sequence.

    Parameters
    ----------
    seq : str
        Input DNA sequence with <mask> token (e.g. "ATCGATCG<mask>ATCG").
    model_id : str
        Hugging Face model ID.
    token : str
        Hugging Face access token with read permissions.
    """
    url = f"https://api-inference.huggingface.co/models/{model_id}"
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.post(url, headers=headers, json={"inputs": seq})

    if response.status_code != 200:
        print(f"❌ API Error {response.status_code}: {response.text}")
        return None
    try:
        return response.json()
    except Exception as e:
        print("⚠️ Response not in JSON format:", e)
        print(response.text[:300])
        return None

In [11]:
masked_seq = "ATCGATCG<mask>ATCG"
model_id = "InstaDeepAI/nucleotide-transformer-500m-human-ref"
result = query_api(masked_seq, model_id, HF_TOKEN)

# ---- Display pedagogic output ----
if result:
    df = pd.DataFrame(result)[["token_str", "score", "sequence"]]
    df.rename(columns={
        "token_str": "Predicted DNA motif (replacing <mask>)",
        "score": "Confidence score",
        "sequence": "Reconstructed sequence"
    }, inplace=True)

    print(f"🔬 Input Sequence: {masked_seq}")
    print(f"🧠 Model: {model_id}\n")

    display(HTML(df.style
        .bar(subset=["Confidence score"], color='#a6cee3')
        .format({
            "Confidence score": "{:.3%}",
            "Predicted DNA motif (replacing <mask>)": lambda x: f"<b style='color:#1f78b4'>{x}</b>",
            "Reconstructed sequence": lambda x: x.replace(x[10:17], f"<b style='color:#33a02c'>{x[10:17]}</b>")
        })
        .set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#f2f2f2'), ('font-weight', 'bold')]}
        ])
        .set_caption("Top-5 model predictions for the masked DNA region")
        .to_html()
    ))

🔬 Input Sequence: ATCGATCG<mask>ATCG
🧠 Model: InstaDeepAI/nucleotide-transformer-500m-human-ref



,Predicted DNA motif (replacing ),Confidence score,Reconstructed sequence
0,AGCCTT,3.342%,ATCGAT C G AGCCTT A T C G
1,TTTCCT,2.425%,ATCGAT C G TTTCCT A T C G
2,CAGCTT,2.418%,ATCGAT C G CAGCTT A T C G
3,CGTCTT,2.347%,ATCGAT C G CGTCTT A T C G
4,CTTTTC,2.220%,ATCGAT C G CTTTTC A T C G


##### Each row shows a possible 6-mer that the model believes fits in the masked position, ordered by confidence. The model reconstructs the complete sequence accordingly.

## Part 4: Hugging Face Inference API (3/3) — Compare Local vs API
**What & Why:** Time a minimal local forward pass vs a remote API call. This illustrates the latency trade‑offs between local GPU and hosted inference.

In [12]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM
import requests
import pandas as pd
from IPython.display import display, HTML
import logging, time

# ------------------------------------------------------------------
# 🔧 1️⃣ Setup
# ------------------------------------------------------------------
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "InstaDeepAI/nucleotide-transformer-500m-human-ref"
print(f"🧠 Using model: {model_id} on {device}\n")

# Load tokenizer once
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# ------------------------------------------------------------------
# 📘 2️⃣ Embedding Extraction (AutoModel)
# ------------------------------------------------------------------
print("📘 Part 1: Embedding Extraction (AutoModel)\n")

embed_model = AutoModel.from_pretrained(model_id, trust_remote_code=True).to(device).eval()

seq_plain = ["ATCGATCGATCG"]
tokens_plain = tokenizer(seq_plain, return_tensors="pt", padding=True, truncation=True).to(device)

with torch.no_grad():
    outputs = embed_model(**tokens_plain)
    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

print(f"✅ Embedding extracted — shape: {embeddings.shape}\n")

# ------------------------------------------------------------------
# ⚙️ 3️⃣ Local MLM Prediction (AutoModelForMaskedLM)
# ------------------------------------------------------------------
print("⚙️ Part 2: Local MLM Prediction (True softmax over full vocab)\n")

# Load the MLM-capable model
mlm_model = AutoModelForMaskedLM.from_pretrained(model_id, trust_remote_code=True).to(device).eval()

# Input sequence with a mask token
seq_masked = ["ATCGATCG<mask>ATCG"]
tokens_masked = tokenizer(seq_masked, return_tensors="pt", padding=True, truncation=True).to(device)

t0 = time.time()
with torch.no_grad():
    outputs = mlm_model(**tokens_masked)
    logits = outputs.logits
t1 = time.time()
local_time = t1 - t0

# Find mask token location and extract logits
mask_token_index = (tokens_masked["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)
mask_logits = logits[mask_token_index]

# Compute probabilities over the full vocab
probs = F.softmax(mask_logits, dim=-1)
top_k = torch.topk(probs, 5, dim=-1)
top_tokens = top_k.indices[0].tolist()
top_scores = top_k.values[0].tolist()

# Build readable table
preds_local = [
    {"Rank": i+1, "Predicted 6-mer": tokenizer.decode([t]).strip(), "Confidence": f"{s*100:.2f}%"}
    for i, (t, s) in enumerate(zip(top_tokens, top_scores))
]

display(HTML(pd.DataFrame(preds_local).style.set_caption("🧩 Local MLM Predictions (Normalized)").to_html()))

# ------------------------------------------------------------------
# ☁️ 4️⃣ Remote Hugging Face API Prediction
# ------------------------------------------------------------------
print("☁️ Part 3: Remote Inference via Hugging Face API\n")

def query_api(seq, model_id, token):
    """Query the Hugging Face Inference API for predictions."""
    url = f"https://api-inference.huggingface.co/models/{model_id}"
    headers = {"Authorization": f"Bearer {token}"}
    r = requests.post(url, headers=headers, json={"inputs": seq})
    if r.status_code != 200:
        print(f"❌ API Error {r.status_code}: {r.text[:300]}")
        return None
    try:
        return r.json()
    except Exception as e:
        print("⚠️ Non-JSON response:", e, r.text[:300])
        return None

t0 = time.time()
api_result = query_api(seq_masked[0], model_id, HF_TOKEN)
t1 = time.time()
api_time = t1 - t0

if api_result:
    preds_api = [
        {"Rank": i+1, "Predicted 6-mer": r["token_str"], "Confidence": f"{r['score']*100:.2f}%"}
        for i, r in enumerate(api_result[:5])
    ]
    display(HTML(pd.DataFrame(preds_api).style.set_caption("☁️ Hugging Face API Predictions").to_html()))
else:
    print("API call failed.")

🧠 Using model: InstaDeepAI/nucleotide-transformer-500m-human-ref on cuda

📘 Part 1: Embedding Extraction (AutoModel)

✅ Embedding extracted — shape: (1, 1280)

⚙️ Part 2: Local MLM Prediction (True softmax over full vocab)



,Rank,Predicted 6-mer,Confidence
0,1,AGCCTT,3.34%
1,2,TTTCCT,2.42%
2,3,CAGCTT,2.42%
3,4,CGTCTT,2.35%
4,5,CTTTTC,2.22%


☁️ Part 3: Remote Inference via Hugging Face API



,Rank,Predicted 6-mer,Confidence
0,1,AGCCTT,3.34%
1,2,TTTCCT,2.42%
2,3,CAGCTT,2.42%
3,4,CGTCTT,2.35%
4,5,CTTTTC,2.22%


In [13]:
import pandas as pd
from IPython.display import display, HTML

# Assume you already have: local_time, api_time, model_id
comparison_data = {
    "Inference Mode": ["Local (GPU/CPU)", "Hugging Face API (Cloud)"],
    "Environment": ["Local runtime / GPU node", "Hugging Face Inference Service"],
    "Input Example": ["ATCGATCGATCG", "ATCGATCG<mask>ATCG"],
    "Avg Runtime (s)": [f"{local_time:.3f}", f"{api_time:.3f}"],
    "Key Advantages": [
        "⚡ Fast, customizable, runs offline",
        "☁️ No setup, accessible anywhere"
    ],
    "Limitations": [
        "Requires installation + GPU/CPU resources",
        "Network latency + model availability constraints"
    ]
}

df = pd.DataFrame(comparison_data)

styled = (
    df.style
    .set_table_styles([
        {"selector": "th", "props": [
            ("background-color", "#f2f2f2"),
            ("font-weight", "bold"),
            ("text-align", "center"),
            ("border", "1px solid #ccc")
        ]},
        {"selector": "td", "props": [
            ("text-align", "center"),
            ("padding", "6px"),
            ("border", "1px solid #eee")
        ]}
    ])
    .set_caption("⚖️ Local vs Hugging Face Inference Comparison")
    .set_properties(subset=["Avg Runtime (s)"], **{"font-weight": "bold", "color": "#1f77b4"})
)

display(HTML(styled.to_html()))

,Inference Mode,Environment,Input Example,Avg Runtime (s),Key Advantages,Limitations
0,Local (GPU/CPU),Local runtime / GPU node,ATCGATCGATCG,0.116,"⚡ Fast, customizable, runs offline",Requires installation + GPU/CPU resources
1,Hugging Face API (Cloud),Hugging Face Inference Service,ATCGATCGATCG,0.410,"☁️ No setup, accessible anywhere",Network latency + model availability constraints


----------------------------------------

## Part 5: HF Datasets + Nucleotide Transformer (Run → Inspect → Answer)

### Objectives (read only)
1. Authenticate to HF Platform
2. Load a genomics dataset from the Hub (already split).
3. Pin a revision for reproducibility.
4. Explore schema, class balance, and sequence stats.
5. Tokenize with a Nucleotide Transformer tokenizer.
6. Fine-tune a classifier head; evaluate.
7. Push artifacts back to the Hub and run hosted inference.

**Prereqs (once)**:

    * pip install -q transformers datasets evaluate huggingface_hub scikit-learn
    * You’ll also need a free Hugging Face account + access token.

### Part 5: HF Datasets (1/7: Load genomics dataset)

Login to Hugging Face Platform by running below from terminal. 

* _You should have account on HuggingFace to proceed with this part._

In [25]:
#!pip install --upgrade huggingface_hub
!source ~/.bashrc
!huggingface-cli login --token "$HF_TOKEN"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `test` has been saved to /biocorelab/BIX/resources/hf_resources/stored_tokens
Your token has been saved to /biocorelab/BIX/resources/hf_resources/token
Login successful.
The current active token is: `test`


In [27]:
from huggingface_hub import notebook_login, HfApi
import transformers, datasets, sys, ipywidgets

print("Transformers:", transformers.__version__)
print("Datasets:", datasets.__version__)
print("Python:", sys.version.split()[0])

api = HfApi()
print("HF user/org:", api.whoami()["name"])

Transformers: 4.49.0
Datasets: 3.6.0
Python: 3.11.13
HF user/org: ullahi


### Part 5: HF Datasets (2/7: Load a pre-split dataset from the Hub)

We will use the dataset _InstaDeepAI/nucleotide_transformer_downstream_tasks_ for this session. It has been released by InstaDeep AI team. 

The dataset collects 18 downstream classification tasks (both binary and multi‐class) that were used to evaluate the Nucleotide Transformer models. Each task corresponds to a genomic regulatory or structural element (for example: promoter detection, enhancer classification, splice-site prediction, histone mark prediction) in human DNA.

In [28]:
from datasets import load_dataset

# Pin a revision for reproducibility (use "main" or a commit hash if you have one)
REV = "main"
ds_all = load_dataset("InstaDeepAI/nucleotide_transformer_downstream_tasks", revision=REV)
ds_all

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sequence', 'name', 'label', 'task'],
        num_rows: 461850
    })
    test: Dataset({
        features: ['sequence', 'name', 'label', 'task'],
        num_rows: 48797
    })
})

### Pick one task and inspect
The dataset holds multiple tasks; we’ll choose one (e.g., "promoter_all"). For sake of testing, we take a subset of the data to speed up training

In [29]:
np.unique(ds_all['train']['task'])

array(['H3', 'H3K14ac', 'H3K36me3', 'H3K4me1', 'H3K4me2', 'H3K4me3',
       'H3K79me3', 'H3K9ac', 'H4', 'H4ac', 'enhancers', 'enhancers_types',
       'promoter_all', 'promoter_no_tata', 'promoter_tata',
       'splice_sites_acceptors', 'splice_sites_all',
       'splice_sites_donors'], dtype='<U22')

In [41]:
from collections import Counter
TASK = "promoter_all"  # try others later!
ds_full = ds_all.filter(lambda ex: ex["task"] == TASK)

# take a subset (15K training and 9K testing sample
ds = DatasetDict({
    "train": ds_full["train"].shuffle(seed=42).select(range(15000)),
    "test":  ds_full["test"].shuffle(seed=42).select(range(5000))
})

{split: len(ds[split]) for split in ds}
print(ds["train"].features)

y = [ex["label"] for ex in ds["train"]]
Counter(y), len(y)

{'sequence': Value(dtype='string', id=None), 'name': Value(dtype='string', id=None), 'label': Value(dtype='int32', id=None), 'task': Value(dtype='string', id=None)}


(Counter({0: 7531, 1: 7469}), 15000)

#### How many sequences are in training set?

#### Sequence length stats 
* Sanity check
* What’s the max sequence length in your sample? _Use this to choose max_length for tokenization_

In [42]:
import numpy as np

lengths = np.array([len(ex["sequence"]) for ex in ds["train"].select(range(min(5000, len(ds["train"]))))])
print(f"Minimum length: {lengths.min()}, Mean Length: {lengths.mean()}, Maximum Length: {lengths.max()}")

Minimum length: 300, Mean Length: 300.0, Maximum Length: 300


#### How many classes does this task have? Is the class balance skewed?

In [43]:
y = [ex["label"] for ex in ds["train"]]
Counter(y), len(y)

(Counter({0: 7531, 1: 7469}), 15000)

In [46]:
from collections import Counter
import pandas as pd
from IPython.display import display, Markdown

# Count labels in train and test
train_counts = Counter([ex["label"] for ex in ds["train"]])
test_counts  = Counter([ex["label"] for ex in ds["test"]])

# Create a summary DataFrame
labels = sorted(set(train_counts.keys()) | set(test_counts.keys()))
df_summary = pd.DataFrame({
    "Label": labels,
    "Train Count": [train_counts.get(lbl, 0) for lbl in labels],
    "Test Count": [test_counts.get(lbl, 0) for lbl in labels],
})
df_summary["Total"] = df_summary["Train Count"] + df_summary["Test Count"]

# Add totals row at the bottom
total_row = pd.DataFrame({
    "Label": ["**Total**"],
    "Train Count": [df_summary["Train Count"].sum()],
    "Test Count": [df_summary["Test Count"].sum()],
    "Total": [df_summary["Total"].sum()]
})
df_summary = pd.concat([df_summary, total_row], ignore_index=True)

# Display as Markdown table
display(Markdown(df_summary.to_markdown(index=False)))

| Label     |   Train Count |   Test Count |   Total |
|:----------|--------------:|-------------:|--------:|
| 0         |          7531 |         2507 |   10038 |
| 1         |          7469 |         2493 |    9962 |
| **Total** |         15000 |         5000 |   20000 |

#### Tokenize using Nucleotide Transformer Tokenizer

In [47]:
from transformers import AutoTokenizer
from datasets import DatasetDict

MODEL_ID = "InstaDeepAI/nucleotide-transformer-v2-50m-3mer-multi-species"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
print(f"Vocabulary size of the tokenizer is {tokenizer.vocab_size}")

MAX_LEN = 512  # adjust if your sequences are much longer
def preprocess(batch):
    toks = tokenizer(batch["sequence"], padding="max_length", truncation=True, max_length=MAX_LEN)
    toks["labels"] = batch["label"]
    return toks

tokenized = DatasetDict({k: v.map(preprocess, batched=True, remove_columns=v.column_names)
                         for k, v in ds.items()})

if "validation" not in tokenized:
    split = tokenized["train"].train_test_split(test_size=0.1, seed=42)
    tokenized = DatasetDict({
        "train": split["train"],
        "validation": split["test"],
        "test": tokenized["test"]
    })

tokenized

Vocabulary size of the tokenizer is 75


Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13500
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
})

In [68]:
tokenizer

EsmTokenizer(name_or_path='InstaDeepAI/nucleotide-transformer-v2-50m-3mer-multi-species', vocab_size=75, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '<unk>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<cls>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [67]:
tokenized['train'][0]['input_ids'][1]

30

#### Create a classifier head on top of NT encoder

In [35]:
from transformers import AutoConfig, AutoModelForSequenceClassification

num_labels = len(set(ds["train"]["label"]))
config = AutoConfig.from_pretrained(MODEL_ID, num_labels=num_labels, trust_remote_code=True)

# AutoModelForSequenceClassification automatically adds a classification head on top of the base model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID, config=config, trust_remote_code=True)

# parameter count
paramInMillions = round(sum(p.numel() for p in model.parameters())/1e6, 2)
print(f"Number of parameters in NT model is {paramInMillions} million")

Number of parameters in NT model is 51.73 million


####  Train (quick run)

In [ ]:
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"]
    }

args = TrainingArguments(
    output_dir="nt_promoter_run",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    num_train_epochs=2,           # keep short for class
    evaluation_strategy="epoch",
    save_strategy="no",
    logging_steps=50,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

train_out = trainer.train()
eval_out = trainer.evaluate()
eval_out

####  Test set evaluation & confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

pred = trainer.predict(tokenized["test"])
y_true = pred.label_ids
y_pred = pred.predictions.argmax(-1)

print(classification_report(y_true, y_pred, digits=4))
confusion_matrix(y_true, y_pred)

### Experimentation
Try to experiment with different options like: 
1. Changing task (TASK)
2. Adjusting MAX_LEN
3. Trying a larger/smaller NT model
4. Tweaking LR/epochs.